In [298]:
with open('data/input_10.txt') as fh:
    inp = list(map(int, fh.read().strip().split(',')))

In [249]:
inp

[83, 0, 193, 1, 254, 237, 187, 40, 88, 27, 2, 255, 149, 29, 42, 100]

In [237]:
class wraplist(object):
    def __init__(self, data):
        self.data = data
        self.n = len(data)
    def __getitem__(self, sl):
        if type(sl) == int:
            return self.data[sl]
        elif sl.start is not None and sl.start == sl.stop:
            return wraplist([])
        stop = None if sl.stop is None or sl.stop == self.n else sl.stop % self.n
        if stop is not None and sl.start >= stop:
            return wraplist(self.data[slice(sl.start, None, sl.step)] + self.data[slice(None, stop, sl.step)])
        else:
            return wraplist(self.data[slice(sl.start, stop, sl.step)])        
    def __setitem__(self, sl, val):
        if type(sl) == int:
            self.data[sl] = val
            return
        elif sl.start is not None and sl.start == sl.stop:
            return
        stop = None if sl.stop is None or sl.stop == self.n else sl.stop % self.n
        if stop is not None and sl.start >= stop:
            self.data[slice(sl.start, None, sl.step)] = val[slice(0, self.n - sl.start)]
            self.data[slice(None, stop, sl.step)] = val[slice(self.n - sl.start, None)]
            return
        else:
            self.data[slice(sl.start, stop, sl.step)] = val
            return
    def __len__(self):
        return len(self.data)
    def __repr__(self):
        return self.data.__repr__()

In [238]:
def revert(wl, start, length):
    #print(start, length, wl[start:start+length], wl[start:start+length][::-1])
    wl[start:start+length] = wl[start:start+length][::-1]

In [315]:
# test case
pos = 0
skip = 0
circle = wraplist(list(range(5)))
for l in list(map(int, "3, 4, 1, 5".split(', '))):
    revert(circle, pos, l)
    print(l, pos, circle)
    pos = (pos + l + skip) % len(circle)
    skip += 1
print(circle, circle[0] * circle[1])

3 0 [2, 1, 0, 3, 4]
4 3 [4, 3, 0, 1, 2]
1 3 [4, 3, 0, 1, 2]
5 1 [3, 4, 2, 1, 0]
[3, 4, 2, 1, 0] 12


In [299]:
def cycle(circle, lengths, pos, skip):
    for l in lengths:
        revert(circle, pos, l)
        # print(l, circle)
        pos = (pos + l + skip) % len(circle)
        skip += 1
    return circle, pos, skip

In [300]:
circle, pos, skip = cycle(wraplist(list(range(256))), inp, 0, 0)

In [301]:
# part 1
circle[0] * circle[1]

20056

In [265]:
with open('data/input_10.txt') as fh:
    inp_str = fh.read().strip()

In [302]:
def densen(sparse_hash):
    dense_hash = []
    for i in range(16):
        offset = i * 16
        val = sparse_hash[offset]
        for j in range(1, 16):
            val ^= sparse_hash[offset + j]
        dense_hash.append(val)
    return dense_hash

def print_hash(dh):
    strings = map(lambda x: "{0:02x}".format(x), dh)
    return "".join(strings)

In [312]:
def getHash(string):
    inp = list(map(ord, string))
    inp += [17, 31, 73, 47, 23]
    circle = wraplist(list(range(256)))
    pos = 0
    skip = 0
    for i in range(64):
        circle, pos, skip = cycle(circle, inp, pos, skip)
    dh = densen(circle)
    return print_hash(dh)

In [313]:
assert(getHash('') == 'a2582a3a0e66e6e86e3812dcb672a272')
assert(getHash('AoC 2017') == '33efeb34ea91902bb2f59c9920caa6cd')
assert(getHash('1,2,3') == '3efbe78a8d82f29979031a4aa0b16a9d')
assert(getHash('1,2,4') == '63960835bcdc130f0b66d7ff4f6a5a8e')

In [314]:
# part 2
getHash(inp_str)

'd9a7de4a809c56bf3a9465cb84392c8e'